# Generate Kerchunk Files from ESGF Data Hosted Through Globus
As a proof of concept, let's create a kerchunk reference file for CMIP6 data stored at Argonne National Laboratory, one data node in the Earth System Grid Federation

## Imports

In [1]:
import xarray as xr
import fsspec
from fsspec.implementations.http import HTTPFileSystem
from kerchunk.hdf import SingleHdf5ToZarr
from pathlib import Path
import os
import ujson

## Read the dataset using xarray - without kerchunk

In [2]:
url  = "https://g-52ba3.fd635.8443.data.globus.org//css03_data/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r9i1p1f1/Omon/epsi100/gn/v20180914/epsi100_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r9i1p1f1_gn_185009-185508.nc"

In [3]:
%%time
ds = xr.open_dataset(f"{url}#mode=bytes")
ds

CPU times: user 180 ms, sys: 61.3 ms, total: 241 ms
Wall time: 56 s


<xarray.Dataset> Size: 34MB
Dimensions:         (y: 332, x: 362, nvertex: 4, time: 60, axis_nbounds: 2)
Coordinates:
    nav_lat         (y, x) float32 481kB ...
    nav_lon         (y, x) float32 481kB ...
    depth           float64 8B ...
  * time            (time) datetime64[ns] 480B 1850-09-16 ... 1855-08-16T12:0...
Dimensions without coordinates: y, x, nvertex, axis_nbounds
Data variables:
    bounds_nav_lon  (y, x, nvertex) float32 2MB ...
    bounds_nav_lat  (y, x, nvertex) float32 2MB ...
    area            (y, x) float32 481kB ...
    time_bounds     (time, axis_nbounds) datetime64[ns] 960B ...
    epsi100         (time, y, x) float32 29MB ...
Attributes: (12/52)
    name:                      /ccc/work/cont003/gencmip6/p86maf/IGCM_OUT/IPS...
    Conventions:               CF-1.7 CMIP-6.2
    creation_date:             2018-08-08T15:28:38Z
    tracking_id:               hdl:21.14100/3855b286-599a-4b76-aea8-da1b4d4d5fc5
    description:               DECK: abrupt-4xCO2
    title:                     IPSL-CM6A-LR model output prepared for CMIP6 /...
    ...                        ...
    model_version:             6.1.5
    realm:                     ocnBgchem
    parent_variant_label:      r1i1p1f1
    history:                   Thu Jan 17 14:46:16 2019: ncrcat /ccc/work/con...
    NCO:                       "4.6.0"
    nco_openmp_thread_number:  1

## Create a Kerchunk Reference File
Let's create a kerchunk reference file, reading data using https and save the kerchunk file locally.

In [4]:
fs = fsspec.filesystem('https') #Filesystem for the ESGF files

fs2 = fsspec.filesystem('')  #local file system to save final jsons to

so = dict() # args to fs.open()
# default_fill_cache=False avoids caching data in between file chunks to lowers memory usage.

def gen_json(file_url):
    with fs.open(f"{file_url}", **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, file_url, inline_threshold=300)
        # inline threshold adjusts the Size below which binary blocks are included directly in the output
        # a higher inline threshold can result in a larger json file but faster loading time
        variable = file_url.split('/')[-1].split('.')[0]
        month = file_url.split('/')[2]
        outf = f'{month}_{variable}.json' #file name to save json to
        with fs2.open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

### Actually generate the kerchunk file using the url

In [5]:
gen_json(url)
!ls *.json

g-52ba3.fd635.8443.data.globus.org_epsi100_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r9i1p1f1_gn_185009-185508.json


## Read the Kerchunk File Back in and Compare with IO time from http
Notice how much faster we can now read in the file!

In [6]:
%%time
ds = xr.open_dataset("reference://",
                     engine="zarr",
                     backend_kwargs={
                         "consolidated": False,
                         "storage_options": {
                             "fo": 'g-52ba3.fd635.8443.data.globus.org_epsi100_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r9i1p1f1_gn_185009-185508.json',
                             "remote_protocol": "https",}
                     }
                    )
ds

CPU times: user 56.1 ms, sys: 12 ms, total: 68.1 ms
Wall time: 2.77 s


<xarray.Dataset> Size: 34MB
Dimensions:         (y: 332, x: 362, nvertex: 4, time: 60, axis_nbounds: 2)
Coordinates:
    depth           float64 8B ...
    nav_lat         (y, x) float32 481kB ...
    nav_lon         (y, x) float32 481kB ...
  * time            (time) datetime64[ns] 480B 1850-09-16 ... 1855-08-16T12:0...
Dimensions without coordinates: y, x, nvertex, axis_nbounds
Data variables:
    area            (y, x) float32 481kB ...
    bounds_nav_lat  (y, x, nvertex) float32 2MB ...
    bounds_nav_lon  (y, x, nvertex) float32 2MB ...
    epsi100         (time, y, x) float32 29MB ...
    time_bounds     (time, axis_nbounds) datetime64[ns] 960B ...
Attributes: (12/52)
    CMIP6_CV_version:          cv=6.2.3.5-2-g63b123e
    Conventions:               CF-1.7 CMIP-6.2
    EXPID:                     abrupt-4xCO2
    NCO:                       "4.6.0"
    activity_id:               CMIP
    branch_method:             standard
    ...                        ...
    sub_experiment_id:         none
    table_id:                  Omon
    title:                     IPSL-CM6A-LR model output prepared for CMIP6 /...
    tracking_id:               hdl:21.14100/3855b286-599a-4b76-aea8-da1b4d4d5fc5
    variable_id:               epsi100
    variant_label:             r9i1p1f1